# Exploratory Analysis of Competitor Reviews

In [1]:
import os
import json
import pandas as pd
from keybert import KeyBERT
from bertopic import BERTopic
import glob

## Load The Dataset

**This can be replaced with a script to automatically pull the reviews**

In [5]:
file_name = "reviews.json"

In [13]:
with open(os.path.join('..', 'data', file_name), 'r') as f:
    reviews = json.load(f)

## Peaking at the Data

In [22]:
reviews['C31DUAL4KDP'][list(reviews['C31DUAL4KDP'].keys())[0]].keys()

dict_keys(['date_info', 'name', 'title', 'content', 'rating'])

### Applying KeyBERT

In [54]:
doc_list = reviews['C31DUAL4KDP'][list(reviews['C31DUAL4KDP'].keys())[2]]['content']
doc = "\n".join(doc_list)
# doc

In [55]:
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 8), stop_words='english')

In [56]:
keywords

[('bought order expand basic small screen laptop dell', 0.7377),
 ('screened brand new lenovo gofanco usb dual displayport', 0.7142),
 ('repeatedly blue screened brand new lenovo gofanco usb', 0.7138),
 ('blue screened brand new lenovo gofanco usb dual', 0.7113),
 ('blue screened brand new lenovo gofanco usb', 0.702)]

### Applying BertTopic

In [57]:
from flair.embeddings import TransformerDocumentEmbeddings

amazon_reviews_zh = TransformerDocumentEmbeddings('activebus/BERT_Review')
# topic_model = BERTopic(embedding_model=roberta)

Some weights of the model checkpoint at activebus/BERT_Review were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
topic_model = BERTopic(embedding_model=amazon_reviews_zh, n_gram_range=(2, 3), min_topic_size=5)
topic_model.fit(doc_list)

In [60]:
topics, _ = topic_model.transform(doc_list)

In [61]:
topic_model.get_topic_info()

,Topic,Count,Name
0,1,27,1_to unplug_display port_my work_unplug it
1,0,12,0_but it_to purchase mini_to put_to put it


In [62]:
topic_model.get_topic(1)

[('to unplug', 0.014460050321665792),
 ('display port', 0.014460050321665792),
 ('my work', 0.011380758546559924),
 ('unplug it', 0.011380758546559924),
 ('works great', 0.011380758546559924),
 ('it in', 0.011380758546559924),
 ('plug and', 0.011380758546559924),
 ('with the', 0.010089981674008814),
 ('further investigation it', 0.008090543075866968),
 ('had to', 0.008090543075866968)]